In [1]:
# Import Statements
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix, f1_score
from scipy import stats
import pandas as pd
import numpy as np
import pickle
import gc
import os

#garbage collector
import gc
gc.enable() #enabling collection

In [2]:
# Importing each csv files
import pandas as pd
orders = pd.read_csv('C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/orders.csv')
products = pd.read_csv('C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/products.csv')
aisles = pd.read_csv('C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/aisles.csv')
departments = pd.read_csv('C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/departments.csv')
order_products_prior = pd.read_csv('C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/order_products__prior.csv')
order_products_train = pd.read_csv('C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/order_products__train.csv')

In [3]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [4]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
# merging orders and prior datasets
prior_orders = pd.merge(orders, order_products_prior, on='order_id', how='inner')
prior_orders = pd.merge(prior_orders, products, how='left', on='product_id')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17


In [6]:
# Replaces NaN with mean of 'days_since_prior_order'
mean = prior_orders['days_since_prior_order'].mean()
mean = np.round(mean,2)
prior_orders.fillna(mean,inplace=True)

In [7]:
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2539329,1,prior,1,2,8,11.1,196,1,0,Soda,77,7
1,2539329,1,prior,1,2,8,11.1,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16
2,2539329,1,prior,1,2,8,11.1,12427,3,0,Original Beef Jerky,23,19
3,2539329,1,prior,1,2,8,11.1,26088,4,0,Aged White Cheddar Popcorn,23,19
4,2539329,1,prior,1,2,8,11.1,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17


<h2>Creating features using user-ids

<h4>max,min and mean number of orders placed by each user

In [8]:
users_df = pd.DataFrame()
users_df['max_number_of_orders'] = prior_orders.groupby('user_id')['order_number'].aggregate('max')
users_df['mean_number_of_orders'] = prior_orders.groupby('user_id')['order_number'].aggregate('mean').astype(np.uint8)
users_df['min_number_of_orders'] = prior_orders.groupby('user_id')['order_number'].aggregate('min')

In [9]:
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders
user_id,,,
1,10,5,1
2,14,8,1
3,12,6,1
4,5,2,1
5,4,2,1


In [10]:
print("Null values - ", users_df.isnull().any().sum())

Null values -  0


<h4>Day of the week the users orders the most.

In [11]:
#https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value
# On which day of the the week did users order the most ?
users_df['max_day_of_week'] = prior_orders.groupby(by=['user_id'])['order_dow'].\
aggregate(lambda x:x.value_counts().index[0])
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week
user_id,,,,
1,10,5,1,4
2,14,8,1,2
3,12,6,1,0
4,5,2,1,4
5,4,2,1,3


<h4>Order hour of the day the users orders the most.

In [12]:
# On which hour of the day did users ordered the most ?
users_df['max_order_hour_of_day'] = prior_orders.groupby(by=['user_id'])['order_dow'].\
aggregate(lambda x: stats.mode(x)[0])
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day
user_id,,,,,
1,10,5,1,4,4
2,14,8,1,2,2
3,12,6,1,0,0
4,5,2,1,4,4
5,4,2,1,3,3


<h4>User sum days since prior order

In [13]:
# User's total/sum of days since prior order
users_df['sum_days_since_prior_order'] = prior_orders.groupby(by=['user_id'])\
['days_since_prior_order'].aggregate('sum')
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order
user_id,,,,,,
1,10,5,1,4,4,1149.5
2,14,8,1,2,2,3050.3
3,12,6,1,0,0,1007.0
4,5,2,1,4,4,259.4
5,4,2,1,3,3,499.1


<h4>Mean number of days since prior orders for each user

In [14]:
# User's average days since prior orders
users_df['mean_days_since_prior_order'] = prior_orders.groupby(by=['user_id'])\
['days_since_prior_order'].aggregate('mean').astype(np.uint8)
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order
user_id,,,,,,,
1,10,5,1,4,4,1149.5,19
2,14,8,1,2,2,3050.3,15
3,12,6,1,0,0,1007.0,11
4,5,2,1,4,4,259.4,14
5,4,2,1,3,3,499.1,13


<h4>User Reordered Ratio

In [15]:
# User's reordered ratio
users_df['reordered_ratio'] = prior_orders.groupby(['user_id'])['reordered'].aggregate("mean")
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio
user_id,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915
2,14,8,1,2,2,3050.3,15,0.476923
3,12,6,1,0,0,1007.0,11,0.625000
4,5,2,1,4,4,259.4,14,0.055556
5,4,2,1,3,3,499.1,13,0.378378


<h4>User total number of orders placed

In [16]:
# total number of orders placed by a user
users_df['total_num_orders'] = prior_orders.groupby(['user_id'])['order_number'].aggregate("sum")
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders
user_id,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343
2,14,8,1,2,2,3050.3,15,0.476923,1567
3,12,6,1,0,0,1007.0,11,0.625000,528
4,5,2,1,4,4,259.4,14,0.055556,52
5,4,2,1,3,3,499.1,13,0.378378,92


<h4>User total products

In [17]:
# Total number of products brought by a user
users_df['user_total_prods'] = prior_orders.groupby(['user_id'])['product_id'].count()
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods
user_id,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59
2,14,8,1,2,2,3050.3,15,0.476923,1567,195
3,12,6,1,0,0,1007.0,11,0.625000,528,88
4,5,2,1,4,4,259.4,14,0.055556,52,18
5,4,2,1,3,3,499.1,13,0.378378,92,37


<h4>User distinct products

In [18]:
# What are the distinct products brought by a User ?
users_df['user_unique_prods'] = prior_orders.groupby(['user_id'])['product_id'].aggregate(lambda x: x.nunique())
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods
user_id,,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59,18
2,14,8,1,2,2,3050.3,15,0.476923,1567,195,102
3,12,6,1,0,0,1007.0,11,0.625000,528,88,33
4,5,2,1,4,4,259.4,14,0.055556,52,18,17
5,4,2,1,3,3,499.1,13,0.378378,92,37,23


<h4>User average basket

In [19]:
# User's average basket size
users_df['user_average_basket'] = users_df['user_total_prods']/users_df['total_num_orders']
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods,user_average_basket
user_id,,,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59,18,0.172012
2,14,8,1,2,2,3050.3,15,0.476923,1567,195,102,0.124442
3,12,6,1,0,0,1007.0,11,0.625000,528,88,33,0.166667
4,5,2,1,4,4,259.4,14,0.055556,52,18,17,0.346154
5,4,2,1,3,3,499.1,13,0.378378,92,37,23,0.402174


<h4>User order starts at

In [20]:
# At what number does the user's order number starts at ? This tells us if the user is an old customer or a new customer 
# of instacart
users_df['user_order_starts_at'] = prior_orders.groupby(['user_id'])['order_id'].aggregate(min)
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods,user_average_basket,user_order_starts_at
user_id,,,,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59,18,0.172012,431534
2,14,8,1,2,2,3050.3,15,0.476923,1567,195,102,0.124442,738281
3,12,6,1,0,0,1007.0,11,0.625000,528,88,33,0.166667,444309
4,5,2,1,4,4,259.4,14,0.055556,52,18,17,0.346154,94891
5,4,2,1,3,3,499.1,13,0.378378,92,37,23,0.402174,157374


<h4>Average number of products brought by user in each purchase

In [21]:
# Lets first get the total products per order
total_prds_per_order = prior_orders.groupby(['user_id','order_id'])['product_id'].aggregate('count').\
to_frame('total_prds_per_order').reset_index()

In [22]:
total_prds_per_order.head()

,user_id,order_id,total_prds_per_order
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6


In [23]:
# Average number of products brought by user in each purchase
users_df['avg_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].mean()
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods,user_average_basket,user_order_starts_at,avg_no_prds_each_purchase
user_id,,,,,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59,18,0.172012,431534,5.900000
2,14,8,1,2,2,3050.3,15,0.476923,1567,195,102,0.124442,738281,13.928571
3,12,6,1,0,0,1007.0,11,0.625000,528,88,33,0.166667,444309,7.333333
4,5,2,1,4,4,259.4,14,0.055556,52,18,17,0.346154,94891,3.600000
5,4,2,1,3,3,499.1,13,0.378378,92,37,23,0.402174,157374,9.250000


In [24]:
# Median number of products brought by user in each purchase
users_df['median_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].median()
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods,user_average_basket,user_order_starts_at,avg_no_prds_each_purchase,median_no_prds_each_purchase
user_id,,,,,,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59,18,0.172012,431534,5.900000,5.5
2,14,8,1,2,2,3050.3,15,0.476923,1567,195,102,0.124442,738281,13.928571,13.5
3,12,6,1,0,0,1007.0,11,0.625000,528,88,33,0.166667,444309,7.333333,7.0
4,5,2,1,4,4,259.4,14,0.055556,52,18,17,0.346154,94891,3.600000,3.0
5,4,2,1,3,3,499.1,13,0.378378,92,37,23,0.402174,157374,9.250000,10.0


In [25]:
# Minimum number of products brought by user in each purchase
users_df['min_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].min()
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods,user_average_basket,user_order_starts_at,avg_no_prds_each_purchase,median_no_prds_each_purchase,min_no_prds_each_purchase
user_id,,,,,,,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59,18,0.172012,431534,5.900000,5.5,4
2,14,8,1,2,2,3050.3,15,0.476923,1567,195,102,0.124442,738281,13.928571,13.5,5
3,12,6,1,0,0,1007.0,11,0.625000,528,88,33,0.166667,444309,7.333333,7.0,5
4,5,2,1,4,4,259.4,14,0.055556,52,18,17,0.346154,94891,3.600000,3.0,2
5,4,2,1,3,3,499.1,13,0.378378,92,37,23,0.402174,157374,9.250000,10.0,5


In [26]:
# Maximum number of products brought by user in each purchase
users_df['max_no_prds_each_purchase'] = total_prds_per_order.groupby(['user_id'])['total_prds_per_order'].max()
users_df.head()

,max_number_of_orders,mean_number_of_orders,min_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods,user_average_basket,user_order_starts_at,avg_no_prds_each_purchase,median_no_prds_each_purchase,min_no_prds_each_purchase,max_no_prds_each_purchase
user_id,,,,,,,,,,,,,,,,,
1,10,5,1,4,4,1149.5,19,0.694915,343,59,18,0.172012,431534,5.900000,5.5,4,9
2,14,8,1,2,2,3050.3,15,0.476923,1567,195,102,0.124442,738281,13.928571,13.5,5,26
3,12,6,1,0,0,1007.0,11,0.625000,528,88,33,0.166667,444309,7.333333,7.0,5,11
4,5,2,1,4,4,259.4,14,0.055556,52,18,17,0.346154,94891,3.600000,3.0,2,7
5,4,2,1,3,3,499.1,13,0.378378,92,37,23,0.402174,157374,9.250000,10.0,5,12


In [27]:
del total_prds_per_order

<h2>Creating features using product-ids

<h4>Number of times each product was purchased

In [28]:
# Total number of times each of the indivisual products were purchased
products_df = pd.DataFrame()
products_df['times_purchased'] = prior_orders.groupby('product_id')['order_number'].count()
products_df.head()

,times_purchased
product_id,
1,1852
2,90
3,277
4,329
5,15


<h4>Total number of times the product was reordered - indicating how much the product was liked

In [29]:
# How many times was each product reordered ? Total number of times each product was reordered
products_df['prod_tot_reorders'] = prior_orders.groupby('product_id')['reordered'].aggregate('sum')
products_df.head()

,times_purchased,prod_tot_reorders
product_id,,
1,1852,1136
2,90,12
3,277,203
4,329,147
5,15,9


<h4>Product Reorder Ratio

In [30]:
# Product's reorder ratio
products_df['prod_reordered_ratio'] = prior_orders.groupby(['product_id'])['reordered'].aggregate("mean")
products_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio
product_id,,,
1,1852,1136,0.613391
2,90,12,0.133333
3,277,203,0.732852
4,329,147,0.446809
5,15,9,0.600000


<h4>Product average cart position(mean, median, max , min , sum, std)

In [31]:
# Statistic features of product's add to cart position
products_df['avg_cart_position'] = prior_orders.groupby(['product_id'])['add_to_cart_order'].aggregate("mean")
products_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position
product_id,,,,
1,1852,1136,0.613391,5.801836
2,90,12,0.133333,9.888889
3,277,203,0.732852,6.415162
4,329,147,0.446809,9.507599
5,15,9,0.600000,6.466667


In [32]:
products_df['median_cart_position'] = prior_orders.groupby(['product_id'])['add_to_cart_order'].aggregate("median")
products_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position
product_id,,,,,
1,1852,1136,0.613391,5.801836,4.0
2,90,12,0.133333,9.888889,8.0
3,277,203,0.732852,6.415162,4.0
4,329,147,0.446809,9.507599,8.0
5,15,9,0.600000,6.466667,6.0


In [33]:
products_df['sum_cart_position'] = prior_orders.groupby(['product_id'])['add_to_cart_order'].aggregate("sum")
products_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position
product_id,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745
2,90,12,0.133333,9.888889,8.0,890
3,277,203,0.732852,6.415162,4.0,1777
4,329,147,0.446809,9.507599,8.0,3128
5,15,9,0.600000,6.466667,6.0,97


In [34]:
products_df['min_cart_position'] = prior_orders.groupby(['product_id'])['add_to_cart_order'].aggregate("min")
products_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position
product_id,,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745,1
2,90,12,0.133333,9.888889,8.0,890,1
3,277,203,0.732852,6.415162,4.0,1777,1
4,329,147,0.446809,9.507599,8.0,3128,1
5,15,9,0.600000,6.466667,6.0,97,3


In [35]:
products_df['max_cart_position'] = prior_orders.groupby(['product_id'])['add_to_cart_order'].aggregate("max")
products_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position
product_id,,,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745,1,74
2,90,12,0.133333,9.888889,8.0,890,1,45
3,277,203,0.732852,6.415162,4.0,1777,1,41
4,329,147,0.446809,9.507599,8.0,3128,1,36
5,15,9,0.600000,6.466667,6.0,97,3,16


In [36]:
products_df['std_cart_position'] = prior_orders.groupby(['product_id'])['add_to_cart_order'].aggregate("std")
products_df.head()

,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position
product_id,,,,,,,,,
1,1852,1136,0.613391,5.801836,4.0,10745,1,74,5.575389
2,90,12,0.133333,9.888889,8.0,890,1,45,7.821670
3,277,203,0.732852,6.415162,4.0,1777,1,41,6.472701
4,329,147,0.446809,9.507599,8.0,3128,1,36,6.861485
5,15,9,0.600000,6.466667,6.0,97,3,16,3.563038


<h4>Product's aisle and department reordered ratio

In [37]:
# Product's aisle reordered ratio
prod_aisle_df =  prior_orders.groupby(['aisle_id'])['reordered'].aggregate("mean").to_frame('aisle_reorder_ratio')
prod_aisle_df = prod_aisle_df.reset_index()
prod_aisle_df.head()

,aisle_id,aisle_reorder_ratio
0,1,0.596597
1,2,0.489326
2,3,0.598007
3,4,0.489533
4,5,0.280627


In [38]:
# Product's department reordered ratio
prod_depart_df =  prior_orders.groupby(['department_id'])['reordered'].aggregate("mean").to_frame('department_reorder_ratio')
prod_depart_df = prod_depart_df.reset_index()
prod_depart_df.head()

,department_id,department_reorder_ratio
0,1,0.541885
1,2,0.407980
2,3,0.628141
3,4,0.649913
4,5,0.569924


In [39]:
# Merging data
merge_data = pd.merge(products,prod_aisle_df,on='aisle_id')
merge_data = pd.merge(merge_data,prod_depart_df,on='department_id')
merge_data.head()

,product_id,product_name,aisle_id,department_id,aisle_reorder_ratio,department_reorder_ratio
0,1,Chocolate Sandwich Cookies,61,19,0.548698,0.57418
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,0.548698,0.57418
2,102,Danish Butter Cookies,61,19,0.548698,0.57418
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,0.548698,0.57418
4,285,Mini Nilla Wafers Munch Pack,61,19,0.548698,0.57418


In [40]:
merge_data = merge_data.drop(['product_name','aisle_id','department_id'],axis=1)
products_df = pd.merge(products_df,merge_data,on='product_id')
products_df.head()

,product_id,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio
0,1,1852,1136,0.613391,5.801836,4.0,10745,1,74,5.575389,0.548698,0.574180
1,2,90,12,0.133333,9.888889,8.0,890,1,45,7.821670,0.152391,0.346721
2,3,277,203,0.732852,6.415162,4.0,1777,1,41,6.472701,0.527615,0.653460
3,4,329,147,0.446809,9.507599,8.0,3128,1,36,6.861485,0.556655,0.541885
4,5,15,9,0.600000,6.466667,6.0,97,3,16,3.563038,0.280627,0.346721


In [41]:
del merge_data,prod_depart_df,prod_aisle_df

<h2> Creating features using user-ids and product-ids

<h4>Number of times User 'A' bought product 'B'

In [92]:
# Number of times each user has brought product 'B'
user_x_product = pd.DataFrame()
user_x_product['times_brought'] = prior_orders.groupby(by=['user_id','product_id'])['order_id'].aggregate('count')

In [93]:
user_x_product = user_x_product.reset_index()
user_x_product.head()

,user_id,product_id,times_brought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


<h4>Number of times User 'A' reordered product 'B'

In [95]:
# How many times did User 'A' reorder product 'B' ?
reordered_count = prior_orders.groupby(by=['user_id','product_id'])['reordered'].\
aggregate('sum').to_frame('uxp_total_reordered')

In [96]:
reordered_count = reordered_count.reset_index()
reordered_count.head()

,user_id,product_id,uxp_total_reordered
0,1,196,9
1,1,10258,8
2,1,10326,0
3,1,12427,9
4,1,13032,2


In [97]:
user_x_product = user_x_product.merge(reordered_count,on=['user_id','product_id'])
del reordered_count

<h4>Finding when the user has bought a product the first time

In [99]:
# For which order number did a user buy each product ?
first_order_num = prior_orders.groupby(by=['user_id','product_id'])['order_number'].\
aggregate('min').to_frame('uxp_first_order_num')

In [100]:
first_order_num = first_order_num.reset_index()
first_order_num.head()

,user_id,product_id,uxp_first_order_num
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [101]:
user_x_product = user_x_product.merge(first_order_num,on=['user_id','product_id'])
del first_order_num

<h4>Average cart position of a product for each user

In [103]:
# Average add to cart position of a user for each product brought
avg_cart_pos_uxp = prior_orders.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('mean').to_frame('uxp_avg_cart_pos')
avg_cart_pos_uxp = avg_cart_pos_uxp.reset_index()
avg_cart_pos_uxp.head()

,user_id,product_id,uxp_avg_cart_pos
0,1,196,1.400000
1,1,10258,3.333333
2,1,10326,5.000000
3,1,12427,3.300000
4,1,13032,6.333333


In [104]:
user_x_product = user_x_product.merge(avg_cart_pos_uxp,on=['user_id','product_id'])
del avg_cart_pos_uxp

<h4>Minimum cart position of a product for each user

In [106]:
# Minimum add to cart position of a user for each product brought
min_cart_pos_uxp = prior_orders.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('min').to_frame('uxp_min_cart_pos')
min_cart_pos_uxp = min_cart_pos_uxp.reset_index()
min_cart_pos_uxp.head()

,user_id,product_id,uxp_min_cart_pos
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,5


In [107]:
user_x_product = user_x_product.merge(min_cart_pos_uxp,on=['user_id','product_id'])
del min_cart_pos_uxp

<h4>Maximum cart position of a product for each user

In [109]:
# Maximum add to cart position of a user for each product brought
max_cart_pos_uxp = prior_orders.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('max').to_frame('uxp_max_cart_pos')
max_cart_pos_uxp = max_cart_pos_uxp.reset_index()
max_cart_pos_uxp.head()

,user_id,product_id,uxp_max_cart_pos
0,1,196,4
1,1,10258,6
2,1,10326,5
3,1,12427,9
4,1,13032,8


In [110]:
user_x_product = user_x_product.merge(max_cart_pos_uxp,on=['user_id','product_id'])
del max_cart_pos_uxp

<h4>Median cart position of a product for each user

In [112]:
# Median add to cart position of a user for each product brought
median_cart_pos_uxp = prior_orders.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('median').to_frame('uxp_median_cart_pos')
median_cart_pos_uxp = median_cart_pos_uxp.reset_index()
median_cart_pos_uxp.head()

,user_id,product_id,uxp_median_cart_pos
0,1,196,1.0
1,1,10258,3.0
2,1,10326,5.0
3,1,12427,2.5
4,1,13032,6.0


In [113]:
user_x_product = user_x_product.merge(median_cart_pos_uxp,on=['user_id','product_id'])
del median_cart_pos_uxp

In [114]:
user_x_product.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos
0,1,196,10,9,1,1.400000,1,4,1.0
1,1,10258,9,8,2,3.333333,2,6,3.0
2,1,10326,1,0,5,5.000000,5,5,5.0
3,1,12427,10,9,1,3.300000,1,9,2.5
4,1,13032,3,2,2,6.333333,5,8,6.0


<h4>Finding when the user has bought a product the last time

In [116]:
last_order_num = prior_orders.groupby(by=['user_id','product_id'])['order_number'].\
aggregate('max').to_frame('uxp_last_order_num')
last_order_num = last_order_num.reset_index()

In [117]:
last_order_num.head()

,user_id,product_id,uxp_last_order_num
0,1,196,10
1,1,10258,10
2,1,10326,5
3,1,12427,10
4,1,13032,10


In [118]:
user_x_product = user_x_product.merge(last_order_num,on=['user_id','product_id'])
del last_order_num

<h4> Average,Median day of the week for purchased product

In [119]:
# Statistic features for day of the week for each purchased product
avg_dow = prior_orders.groupby(by=['user_id','product_id'])['order_dow'].\
aggregate('mean').to_frame('uxp_avg_dow')
avg_dow = avg_dow.reset_index()

In [120]:
avg_dow.head()

,user_id,product_id,uxp_avg_dow
0,1,196,2.500000
1,1,10258,2.555556
2,1,10326,4.000000
3,1,12427,2.500000
4,1,13032,2.666667


In [121]:
median_dow = prior_orders.groupby(by=['user_id','product_id'])['order_dow'].\
aggregate('median').to_frame('uxp_median_dow')
median_dow = median_dow.reset_index()

In [122]:
median_dow.head()

,user_id,product_id,uxp_median_dow
0,1,196,2.5
1,1,10258,3.0
2,1,10326,4.0
3,1,12427,2.5
4,1,13032,3.0


In [123]:
user_x_product = user_x_product.merge(avg_dow,on=['user_id','product_id'])
user_x_product = user_x_product.merge(median_dow,on=['user_id','product_id'])
del avg_dow,median_dow

<h4> Average,Median hour of the day for purchased product

In [124]:
# statistic features for hour of the day for each purchased product
avg_hour_of_the_day = prior_orders.groupby(by=['user_id','product_id'])['order_hour_of_day'].\
aggregate('mean').to_frame('uxp_avg_hour_of_the_day')
avg_hour_of_the_day = avg_hour_of_the_day.reset_index()

In [125]:
avg_hour_of_the_day.head()

,user_id,product_id,uxp_avg_hour_of_the_day
0,1,196,10.300000
1,1,10258,10.555556
2,1,10326,15.000000
3,1,12427,10.300000
4,1,13032,8.000000


In [126]:
median_hour_of_the_day = prior_orders.groupby(by=['user_id','product_id'])['order_hour_of_day'].\
aggregate('median').to_frame('uxp_median_hour_of_the_day')
median_hour_of_the_day = median_hour_of_the_day.reset_index()

In [127]:
median_hour_of_the_day.head()

,user_id,product_id,uxp_median_hour_of_the_day
0,1,196,8.5
1,1,10258,9.0
2,1,10326,15.0
3,1,12427,8.5
4,1,13032,8.0


In [128]:
user_x_product = user_x_product.merge(avg_hour_of_the_day,on=['user_id','product_id'])
user_x_product = user_x_product.merge(median_hour_of_the_day,on=['user_id','product_id'])
del avg_hour_of_the_day,median_hour_of_the_day

<h4> User-Product Order Rate - How frequent a product is brought by a user

In [136]:
#Total orders
total_orders = prior_orders.groupby('user_id')['order_number'].max().\
to_frame('total_orders').reset_index()
total_orders.head()

,user_id,total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [137]:
total_orders_per_prod = prior_orders.groupby(by=['user_id','product_id'])['order_number'].count().\
to_frame('total_orders_per_prod').reset_index()
total_orders_per_prod.head()

,user_id,product_id,total_orders_per_prod
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [138]:
#merging both data frames to calculate order rate for user-product
uxp_order_rate = pd.merge(total_orders,total_orders_per_prod,on='user_id')
uxp_order_rate.head()

,user_id,total_orders,product_id,total_orders_per_prod
0,1,10,196,10
1,1,10,10258,9
2,1,10,10326,1
3,1,10,12427,10
4,1,10,13032,3


In [139]:
uxp_order_rate['uxp_order_rate'] = uxp_order_rate['total_orders_per_prod'] / uxp_order_rate['total_orders']
uxp_order_rate.head()

,user_id,total_orders,product_id,total_orders_per_prod,uxp_order_rate
0,1,10,196,10,1.0
1,1,10,10258,9,0.9
2,1,10,10326,1,0.1
3,1,10,12427,10,1.0
4,1,10,13032,3,0.3


In [140]:
# drop total_orders and total orders per prod column to avoid collinearity between features
uxp_order_rate = uxp_order_rate.drop(['total_orders','total_orders_per_prod'],axis=1)
uxp_order_rate.head()

,user_id,product_id,uxp_order_rate
0,1,196,1.0
1,1,10258,0.9
2,1,10326,0.1
3,1,12427,1.0
4,1,13032,0.3


In [141]:
user_x_product = user_x_product.merge(uxp_order_rate,on=['user_id','product_id'])
del uxp_order_rate

<h4>Sum across orders of the position in User A’s cart that Item B falls into.

In [142]:
# Sum across orders of the position in User A’s cart that Item B falls into.
sum_cart_pos_uxp = prior_orders.groupby(by=['user_id','product_id'])['add_to_cart_order'].\
aggregate('sum').to_frame('uxp_sum_cart_pos')
sum_cart_pos_uxp = sum_cart_pos_uxp.reset_index()
sum_cart_pos_uxp.head()

,user_id,product_id,uxp_sum_cart_pos
0,1,196,14
1,1,10258,30
2,1,10326,5
3,1,12427,33
4,1,13032,19


In [143]:
user_x_product = user_x_product.merge(sum_cart_pos_uxp,on=['user_id','product_id'])
del sum_cart_pos_uxp

<h4> Mininum user-item order days

In [145]:
# Mininum user-item order days
min_order_days_uxp = prior_orders.groupby(by=['user_id','product_id'])['days_since_prior_order'].\
aggregate('min').to_frame('uxp_min_order_days')
min_order_days_uxp = min_order_days_uxp.reset_index()
min_order_days_uxp.head()

,user_id,product_id,uxp_min_order_days
0,1,196,0.0
1,1,10258,0.0
2,1,10326,28.0
3,1,12427,0.0
4,1,13032,15.0


In [146]:
user_x_product = user_x_product.merge(min_order_days_uxp,on=['user_id','product_id'])
del min_order_days_uxp

<h4> Maximum user-item order days

In [147]:
# Maximum user-item order days
max_order_days_uxp = prior_orders.groupby(by=['user_id','product_id'])['days_since_prior_order'].\
aggregate('max').to_frame('uxp_max_order_days')
max_order_days_uxp = max_order_days_uxp.reset_index()
max_order_days_uxp.head()

,user_id,product_id,uxp_max_order_days
0,1,196,30.0
1,1,10258,30.0
2,1,10326,28.0
3,1,12427,30.0
4,1,13032,30.0


<h4> Median user-item order days

In [148]:
# Median user-item order days
median_order_days_uxp = prior_orders.groupby(by=['user_id','product_id'])['days_since_prior_order'].\
aggregate('median').to_frame('uxp_median_order_days')
median_order_days_uxp = median_order_days_uxp.reset_index()
median_order_days_uxp.head()

,user_id,product_id,uxp_median_order_days
0,1,196,19.5
1,1,10258,20.0
2,1,10326,28.0
3,1,12427,19.5
4,1,13032,20.0


In [149]:
user_x_product = user_x_product.merge(max_order_days_uxp,on=['user_id','product_id'])
user_x_product = user_x_product.merge(median_order_days_uxp,on=['user_id','product_id'])
del median_order_days_uxp,max_order_days_uxp

<h4>User Product Reorder Ratio

In [150]:
# User Product Reorder Ratio
uxp_reorder_ratio = prior_orders.groupby(by=['user_id','product_id'])['reordered'].aggregate("mean")\
.to_frame('uxp_reorder_ratio')
uxp_reorder_ratio = uxp_reorder_ratio.reset_index()
uxp_reorder_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [151]:
user_x_product = user_x_product.merge(uxp_reorder_ratio,on=['user_id','product_id'])
del uxp_reorder_ratio

<h2>Prepare order streak

In [87]:
# https://www.kaggle.com/mmueller/order-streaks-feature
# The sign indicates the type of the streak (ordered vs not ordered). So for example "-3" means: 
# product has not been ordered the  last 3 previous orders and there is either no 
# 4th previous order or the product has been ordered back then.
"""
@author: Faron
"""
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

'''
Calculates (user, product) order_streak for the last n orders.

- abs(order_streak) is length of streak
- sgn(order_streak) encodes type of streak (non-ordered vs ordered)
'''

DATA_DIR = "C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/"
PRIOR_FILE = "order_products__prior"
ORDERS_FILE = "orders"


def load_input_data():
    PATH = "{}{}{}".format(DATA_DIR, PRIOR_FILE, ".csv")
    prior = pd.read_csv(PATH, dtype={'order_id': np.int32,
                                     'product_id': np.uint16,
                                     'add_to_cart_order': np.int16,
                                     'reordered': np.int8})

    PATH = "{}{}{}".format(DATA_DIR, ORDERS_FILE, ".csv")
    orders = pd.read_csv(PATH, dtype={'order_id': np.int32,
                                      'user_id': np.int64,
                                      'order_number': np.int16,
                                      'order_dow': np.int8,
                                      'order_hour_of_day': np.int8,
                                      'days_since_prior_order': np.float32})
    return prior, orders


def apply_parallel(df_groups, _func):
    nthreads = multiprocessing.cpu_count() >> 1
    print("nthreads: {}".format(nthreads))

    res = Parallel(n_jobs=nthreads)(delayed(_func)(grp.copy()) for _, grp in df_groups)
    return pd.concat(res)


def add_order_streak(df):
    tmp = df.copy()
    tmp.user_id = 1

    UP = tmp.pivot(index="product_id", columns='order_number').fillna(-1)
    UP.columns = UP.columns.droplevel(0)

    x = np.abs(UP.diff(axis=1).fillna(2)).values[:, ::-1]
    df.set_index("product_id", inplace=True)
    df['order_streak'] = np.multiply(np.argmax(x, axis=1) + 1, UP.iloc[:, -1])
    df.reset_index(drop=False, inplace=True)
    return df


if __name__ == '__main__':
    prior, orders = load_input_data()

    print("orders: {}".format(orders.shape))
    print("take only recent 5 orders per user:")
    orders = orders.groupby(['user_id']).tail(5 + 1)
    print("orders: {}".format(orders.shape))

    prior = orders.merge(prior, how='inner', on="order_id")
    prior = prior[['user_id', 'product_id', 'order_number']]
    print("prior: {}".format(prior.shape))

    user_groups = prior.groupby('user_id')
    s = datetime.now()
    df = apply_parallel(user_groups, add_order_streak)
    e = datetime.now()
    print("time elapsed: {}".format(e - s))

    df = df.drop("order_number", axis=1).drop_duplicates().reset_index(drop=True)
    df = df[['user_id', 'product_id', 'order_streak']]
    print(df.head(n=10))
    df.to_csv("C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/order_streaks.csv", index=False)
    print("order_streaks.csv has been written")

orders: (3421083, 7)
take only recent 5 orders per user:
orders: (1169692, 7)
prior: (9863821, 3)
nthreads: 4
time elapsed: 0:12:22.238890
   user_id  product_id  order_streak
0        1         196           5.0
1        1       12427           5.0
2        1       10258           5.0
3        1       25133           5.0
4        1       13032           1.0
5        1       46149           3.0
6        1       49235          -1.0
7        1       39657           1.0
8        1       38928           1.0
9        1       35951           1.0
order_streaks.csv has been written


In [153]:
order_streaks = pd.read_csv("C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/order_streaks.csv")

In [154]:
order_streaks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6714245 entries, 0 to 6714244
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   user_id       int64  
 1   product_id    int64  
 2   order_streak  float64
dtypes: float64(1), int64(2)
memory usage: 153.7 MB


In [160]:
# Merge order streaks with uxp dataframe
user_x_product = user_x_product.merge(order_streaks,on=['user_id','product_id'],how='left')

In [161]:
# uxp head
user_x_product.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,...,uxp_median_dow,uxp_avg_hour_of_the_day,uxp_median_hour_of_the_day,uxp_order_rate,uxp_sum_cart_pos,uxp_min_order_days,uxp_max_order_days,uxp_median_order_days,uxp_reorder_ratio,order_streak
0,1,196,10,9,1,1.400000,1,4,1.0,10,...,2.5,10.300000,8.5,1.0,14,0.0,30.0,19.5,0.900000,5.0
1,1,10258,9,8,2,3.333333,2,6,3.0,10,...,3.0,10.555556,9.0,0.9,30,0.0,30.0,20.0,0.888889,5.0
2,1,10326,1,0,5,5.000000,5,5,5.0,5,...,4.0,15.000000,15.0,0.1,5,28.0,28.0,28.0,0.000000,NaN
3,1,12427,10,9,1,3.300000,1,9,2.5,10,...,2.5,10.300000,8.5,1.0,33,0.0,30.0,19.5,0.900000,5.0
4,1,13032,3,2,2,6.333333,5,8,6.0,10,...,3.0,8.000000,8.0,0.3,19,15.0,30.0,20.0,0.666667,1.0


In [162]:
# uxp dataframe information
user_x_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 21 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   user_id                     int64  
 1   product_id                  int64  
 2   times_brought               int64  
 3   uxp_total_reordered         int64  
 4   uxp_first_order_num         int64  
 5   uxp_avg_cart_pos            float64
 6   uxp_min_cart_pos            int64  
 7   uxp_max_cart_pos            int64  
 8   uxp_median_cart_pos         float64
 9   uxp_last_order_num          int64  
 10  uxp_avg_dow                 float64
 11  uxp_median_dow              float64
 12  uxp_avg_hour_of_the_day     float64
 13  uxp_median_hour_of_the_day  float64
 14  uxp_order_rate              float64
 15  uxp_sum_cart_pos            int64  
 16  uxp_min_order_days          float64
 17  uxp_max_order_days          float64
 18  uxp_median_order_days       float64
 19  uxp_reorder_ratio  

In [163]:
users_df.to_pickle("C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/users_df.pkl")
products_df.to_pickle("C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/products_df.pkl")
user_x_product.to_pickle("C:/Users/Srinidhi.Karjol/ML/case studies/mycasestudies/instacart-market-basket-analysis/uxp_df.pkl")